# Assignment 2: Analysing missing data in F1 Dataset

**Objectives:**

* Uncover patterns of missing data in datasets
* See how missing data can affect the distribution of variables
* Use and evaluate imputation techniques

## Setup

In [3]:
# %matplotlib inline

### Load Relevant Packages

In [5]:
from pyspark.sql.types import IntegerType
from pyspark.sql import functions as F
import pandas as pd
import numpy as np
from numpy.random import rand
from matplotlib import pyplot as plt
import seaborn as sns
import sklearn
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import KNNImputer
from sklearn.metrics import confusion_matrix

### Mount S3 Bucket to obtain data

* Provide AWS Access Keys for authentication
* **DELETE ACCESS KEYS IMMEDIATELY AFTER MOUNTING**
* Provide AWS Bucket Name and mount name
* Expect successful mount - line 8 should produce paths to be used to get data

In [7]:
# Uncomment, insert credentials, and run to mount
#ACCESS_KEY = ''
# Encode the Secret Key as that can contain '/'
#SECRET_KEY = ''.replace("/", "%2F")
#AWS_BUCKET_NAME_RAW = 'ne-gr5069'
#MOUNT_NAME_RAW = 'ne-gr5069'

#dbutils.fs.mount('s3a://%s:%s@%s' % (ACCESS_KEY, SECRET_KEY, AWS_BUCKET_NAME_RAW),
#                 '/mnt/%s' % MOUNT_NAME_RAW)
#display(dbutils.fs.ls('/mnt/%s' % MOUNT_NAME_RAW))

## Mount S3 Bucket to Write Data

In [9]:
AWS_BUCKET_NAME_PROC = 'xql2001-gr5069'
MOUNT_NAME_PROC = 'xql2001-gr5069'

dbutils.fs.mount('s3a://%s:%s@%s' % (ACCESS_KEY, SECRET_KEY, AWS_BUCKET_NAME_PROC),
                 '/mnt/%s' % MOUNT_NAME_PROC)
display(dbutils.fs.ls('/mnt/%s' % MOUNT_NAME_PROC))

path,name,size
dbfs:/mnt/xql2001-gr5069/interim/,interim/,0
dbfs:/mnt/xql2001-gr5069/processed/,processed/,0


### Define functions to be used throughout

In [11]:
def fill_dummy_values(df, scaling_factor):
  # Since Matplotlib does not plot missing values, we would have to create 
  # a function that fills dummy variables into the null values of the
  # dataframe
    #create copy of dataframe
    df_dummy = df.copy(deep = True)
    # Iterate over each column
    for col in df_dummy:
        if df_dummy.dtypes[col] != np.object:
          #get column, column missing values and range
          col = df_dummy[col]
          col_null = col.isnull()
          num_nulls = col_null.sum()
          col_range = col.max() - col.min()
        
          #Shift and scale dummy values
          dummy_values = (rand(num_nulls) - 2)
          dummy_values = dummy_values * scaling_factor * col_range + col.min()
        
          #Return dummy values
          col[col_null] = dummy_values
    return df_dummy

def cols_to_int_type(df, col_list):
  # :::::::::::: DESCRIPTION
  # This function is used to change a set of columns in a dataframe to an
  # integer type by looping through through columnsprovided as a list
  #
  # Functionalising this because many of the dfs in the F1 data should be
  # integers, but every variable is imported as string
  #
  # ::::::::: INPUTS
  # 1. df - the dataframe with columns to be changed to int. Should be a 
  #    pyspark.sql dataframe object
  # 2. col_list - a list of strings - each the name of a column in the df
  #    that is to be changed to an integer datatype
  #
  # ::::::::: OUTPUT
  # The dataframe entered as an argument, but with the desired columns
  # cast to the datatype Integer
  #
  for colname in col_list:
    df = df.withColumn(colname, df[colname].cast(IntegerType()))
  return(df)


def encode(data):
    '''function to encode non-null data and replace it in the original data'''
    #retains only non-null values
    nonulls = np.array(data.dropna())
    #reshapes the data for encoding
    impute_reshape = nonulls.reshape(-1,1)
    #encode date
    impute_ordinal = encoder.fit_transform(impute_reshape)
    #Assign back encoded values to non-null values
    data.loc[data.notnull()] = np.squeeze(impute_ordinal)
    return data
  
def RMSE(predict, target):
  #Calculate RMSE
  return np.sqrt(((predict - target) ** 2).mean())

### Load in missing data dataset

In [13]:
df_missing = spark.read.csv('/mnt/ne-gr5069/raw/df_missing.csv', header = True)
display(df_missing)

_c0,driverId,driverRef,number,code,forename,surname,dob,nationality,url,age,raceId,lap,position,time,milliseconds
1,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,37,17,2,1:37.663,97663
2,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,37,18,2,1:37.579,97579
3,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,37,19,1,1:37.720,97720
4,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,37,20,1,1:40.973,100973
5,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,37,21,3,2:00.933,120933
6,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,37,22,3,1:36.701,96701
7,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,37,23,2,1:37.257,97257
8,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,37,24,2,1:37.907,97907
9,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,37,25,2,1:38.180,98180
10,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,37,26,2,1:37.822,97822


In [14]:
display(df_missing.summary())

summary,_c0,driverId,driverRef,number,code,forename,surname,dob,nationality,url,age,raceId,lap,position,time,milliseconds
count,472504,472504,472504,472504,472504,472504,472504,472504,472504,472504,472504,472504,472504,472504,472504,472504
mean,236252.5,229.7737775764861,null,25.328412820757087,null,null,null,null,null,null,36.98124702198625,480.07822156002914,29.871804259858116,9.651368030746829,null,94899.58912224017
stddev,136400.3001340784,345.3821632369074,null,24.985651553593236,null,null,null,null,null,null,7.251128022983076,402.0313510566064,18.38969559962932,5.562433907092902,null,66244.43117416694
min,1,1,albers,10,ALB,Adrian,Albers,1958-02-18,American,http://en.wikipedia.org/wiki/Adrian_Sutil,20,1,1,1,15:25.994,100000
25%,118129.0,14.0,null,8.0,null,null,null,null,null,null,32.0,112.0,14.0,5.0,null,81931.0
50%,236205.0,30.0,null,19.0,null,null,null,null,null,null,38.0,228.0,29.0,9.0,null,90364.0
75%,354341.0,807.0,null,28.0,null,null,null,null,null,null,43.0,911.0,44.0,14.0,null,102028.0
max,99999,9,zonta,\N,\N,Zsolt,van der Garde,1999-11-13,Venezuelan,http://en.wikipedia.org/wiki/Zsolt_Baumgartner,NA,999,9,9,NA,NA


### Change datatype of appropriate columns to int

In [16]:
df_missing_int_colname = ['_c0',
                          'driverId',
                          'number',
                          'age',
                          'raceId',
                          'lap',
                          'position',
                          'milliseconds']

df_missing = cols_to_int_type(df_missing, df_missing_int_colname)
display(df_missing)

_c0,driverId,driverRef,number,code,forename,surname,dob,nationality,url,age,raceId,lap,position,time,milliseconds
1,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,37,17,2,1:37.663,97663
2,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,37,18,2,1:37.579,97579
3,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,37,19,1,1:37.720,97720
4,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,37,20,1,1:40.973,100973
5,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,37,21,3,2:00.933,120933
6,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,37,22,3,1:36.701,96701
7,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,37,23,2,1:37.257,97257
8,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,37,24,2,1:37.907,97907
9,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,37,25,2,1:38.180,98180
10,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,37,26,2,1:37.822,97822


### Identify and Isolate Missing values

*There are missing values in other rows as well, like driver code (in the last
assignment), but here our focus is on the 3 identified for the assignment -
`milliseconds`, `nationality`, and `age`*

I downloaded the raw data and the datatype transformed data from above and had
a look in Excel (shameless, I know, but it's just very comfortable to use 
Excel when one needs to just scroll through a table and have a look).

Missing values in the raw data were coded as "NA". This is still true for the
'nationality' column, which is still string type. For the age and milliseconds
columns, transforming the data to integer type caused non number values to
become blank (e.g. from NA to a blank).

The following cells are meant to further explore the data after transformation.
The main goal is to see if we've managed to identify the pattern of all the
missing or invalid values (all missing values are coded NA, not NULL or
NaN, and no more stray missing or invalid vals in age and milliseconds)

In [18]:
display(df_missing.summary())

summary,_c0,driverId,driverRef,number,code,forename,surname,dob,nationality,url,age,raceId,lap,position,time,milliseconds
count,472504,472504,472504,230938,472504,472504,472504,472504,472504,472504,411348,472504,472504,472504,472504,425253
mean,236252.5,229.7737775764861,null,25.328412820757087,null,null,null,null,null,null,36.98124702198625,480.07822156002914,29.871804259858116,9.651368030746829,null,94899.58912224017
stddev,136400.3001340784,345.3821632369074,null,24.985651553593236,null,null,null,null,null,null,7.251128022983076,402.0313510566064,18.38969559962932,5.562433907092902,null,66244.43117416694
min,1,1,albers,2,ALB,Adrian,Albers,1958-02-18,American,http://en.wikipedia.org/wiki/Adrian_Sutil,20,1,1,1,15:25.994,66957
25%,118129,14,null,8,null,null,null,null,null,null,32,112,14,5,null,81931
50%,236205,30,null,19,null,null,null,null,null,null,38,228,29,9,null,90364
75%,354341,807,null,28,null,null,null,null,null,null,43,911,44,14,null,102028
max,472504,848,zonta,99,\N,Zsolt,van der Garde,1999-11-13,Venezuelan,http://en.wikipedia.org/wiki/Zsolt_Baumgartner,48,1030,78,24,NA,7507547


In [19]:
df_missing_unique_nationality = df_missing.groupby('nationality')\
  .count()

display(df_missing_unique_nationality)

nationality,count
Mexican,11498
Finnish,35924
Swiss,2334
Thai,1117
NA,47112
Indian,2502
Indonesian,554
Irish,570
Argentine,1659
Polish,4858


**Age and millisecond data:**
It seems that the change to integer type has led to some missing data being
identified. summary() now produces 411348 entries for age and 425253 entries
for milliseconds, both less than the total number of rows, 472504.

The average, min, and max values of age and milliseconds appear to be
reasonable for F1 drivers and laps. I think it is reasonable to conclude any
data left in the column are valid ages and laptimes.

**Nationality data:**

It appears the only values that NA is the only tag for missing values.
Everything else seems to be a valid nationality.

Although, the overall DF summary suggests that there was no missing data in the
nationality set, that's likely because "NA" was considered a valid string.

The groupby operation above shows that there were 47112 NA values, which should
be all missing values in the column. This corresponds to 425392 valid entries.

## 1. Three variables, `milliseconds`, `nationality` and `age` have different missingness patterns. Use exploratory data analysis to determine the missingness pattern on each variable, and explain how you reached that conclusion.

### Age
Will first try to filter for just the missing data and look for patterns

In [23]:
df_age_na = df_missing.where(F.col('age').isNull())
display(df_age_na)

_c0,driverId,driverRef,number,code,forename,surname,dob,nationality,url,age,raceId,lap,position,time,milliseconds
226223,30,michael_schumacher,null,MSC,Michael,Schumacher,1969-01-03,German,http://en.wikipedia.org/wiki/Michael_Schumacher,null,81,18,6,1:23.263,83263
226224,30,michael_schumacher,null,MSC,Michael,Schumacher,1969-01-03,German,http://en.wikipedia.org/wiki/Michael_Schumacher,null,96,27,1,1:30.867,90867
226225,30,michael_schumacher,null,MSC,Michael,Schumacher,1969-01-03,German,http://en.wikipedia.org/wiki/Michael_Schumacher,null,96,16,1,1:31.407,91407
226226,30,michael_schumacher,null,MSC,Michael,Schumacher,1969-01-03,German,http://en.wikipedia.org/wiki/Michael_Schumacher,null,340,55,10,1:56.373,116373
226227,30,michael_schumacher,null,MSC,Michael,Schumacher,1969-01-03,German,http://en.wikipedia.org/wiki/Michael_Schumacher,null,173,34,2,1:41.073,101073
226228,30,michael_schumacher,null,MSC,Michael,Schumacher,1969-01-03,German,http://en.wikipedia.org/wiki/Michael_Schumacher,null,96,11,3,1:31.545,91545
226229,30,michael_schumacher,null,MSC,Michael,Schumacher,1969-01-03,German,http://en.wikipedia.org/wiki/Michael_Schumacher,null,129,13,2,1:10.454,70454
226230,30,michael_schumacher,null,MSC,Michael,Schumacher,1969-01-03,German,http://en.wikipedia.org/wiki/Michael_Schumacher,null,129,14,2,1:09.914,69914
226231,30,michael_schumacher,null,MSC,Michael,Schumacher,1969-01-03,German,http://en.wikipedia.org/wiki/Michael_Schumacher,null,340,38,11,1:49.066,109066
226232,30,michael_schumacher,null,MSC,Michael,Schumacher,1969-01-03,German,http://en.wikipedia.org/wiki/Michael_Schumacher,null,134,52,3,1:15.779,75779


In [24]:
# Check if this is all the missing values
display(df_age_na.summary())

summary,_c0,driverId,driverRef,number,code,forename,surname,dob,nationality,url,age,raceId,lap,position,time,milliseconds
count,61156,61156,61156,0,61156,61156,61156,61156,61156,61156,0,61156,61156,61156,61156,58583
mean,288821.4984956505,53.00956570083066,null,null,null,null,null,null,null,null,null,204.3344888481915,29.856596245666818,8.046585780626595,null,91961.53795469676
stddev,34336.11243575735,15.058211199776613,null,null,null,null,null,null,null,null,null,126.41691291549411,18.933034311167777,5.066620639823924,null,38600.2245319102
min,226223,30,alesi,null,MSC,Alessandro,Alesi,1958-02-18,Austrian,http://en.wikipedia.org/wiki/Alex_Zanardi,null,53,1,1,19:29.677,68337
25%,271246,44,null,null,null,null,null,null,null,null,null,156,14,4,null,82126
50%,296776,56,null,null,null,null,null,null,null,null,null,192,28,8,null,87361
75%,315801,64,null,null,null,null,null,null,null,null,null,220,44,12,null,97727
max,339823,86,zanardi,null,\N,Ukyo,Zanardi,1970-05-22,NA,http://en.wikipedia.org/wiki/Ukyo_Katayama,null,879,78,24,NA,7502081


In [25]:
# Looks like we're good
61156 + 411348

Out[10]: 472504

By clicking the sort buttons in the filtered data table and looking at the 
summary, I think the missingness pattern is related to the drivers, for 
the following reasons:

1. Other variables appear to have the full range of variation. Only raceId and
driver related variables like driverId appear to have truncated variation. The
filtered dataset only contains race numbers 50-879 as opposed to 1-1030, and 
drivers 30-86 as opposed to 1-848.

2. Sorting the table with only missing age values up and down in databricks
does not do anything for driver related variables. Only Michael Schumacher 
is reflected.

I think the restriction in the range of raceIds might be related to the
drivers. As drivers 30-86 probably don't have careers that span the whole 
dataset, they aren't going to have all the raceIds reflected in their data.
Furthermore, looking at the data in Excel (again!) I realised that all Michael
Schumacher's age data is missing, not just those for specific races. So I'm 
going to look into driverIds first and see if that resolves the issue, then
look at races if it does not.

In [27]:
df_age_na_driverIds = df_age_na.groupby('driverId')\
  .count()
display(df_age_na_driverIds)

driverId,count
85,119
65,3740
78,277
81,355
44,5891
86,139
57,4957
64,3082
55,4870
49,5831


In [28]:
list_age_na_drivers = df_age_na_driverIds.toPandas()['driverId'].tolist()

In [29]:
df_age_na_replicated = df_missing\
  .where(F.col('driverId').isin(list_age_na_drivers))

display(df_age_na_replicated.summary())

summary,_c0,driverId,driverRef,number,code,forename,surname,dob,nationality,url,age,raceId,lap,position,time,milliseconds
count,61156,61156,61156,0,61156,61156,61156,61156,61156,61156,0,61156,61156,61156,61156,58583
mean,288821.4984956505,53.00956570083066,null,null,null,null,null,null,null,null,null,204.3344888481915,29.856596245666818,8.046585780626595,null,91961.53795469676
stddev,34336.11243575735,15.058211199776613,null,null,null,null,null,null,null,null,null,126.41691291549411,18.933034311167777,5.066620639823924,null,38600.2245319102
min,226223,30,alesi,null,MSC,Alessandro,Alesi,1958-02-18,Austrian,http://en.wikipedia.org/wiki/Alex_Zanardi,null,53,1,1,19:29.677,68337
25%,271246,44,null,null,null,null,null,null,null,null,null,156,14,4,null,82126
50%,296776,56,null,null,null,null,null,null,null,null,null,192,28,8,null,87361
75%,315801,64,null,null,null,null,null,null,null,null,null,220,44,12,null,97727
max,339823,86,zanardi,null,\N,Ukyo,Zanardi,1970-05-22,NA,http://en.wikipedia.org/wiki/Ukyo_Katayama,null,879,78,24,NA,7502081


Success! By getting the driverIds within df_age_na dataframe, then
filtering the overall missing dataset to include entries with only these
driverIds, we successfully replicated the df_age_na dataframe in the
dataframe df_age_na_replicated, as can be seen by:

1. Counts are the same for both these dataframes
2. Age is missing for the entire replicated dataframe.
3. Descriptive stats for variables like lap, position, and milliseconds
 are identical.
 
 We can thus conclude that the missingness pattern of age in df_missing
 is that age is missing for the drivers with driverIds contained within
 `list_age_na_drivers`.
 
 **Note:** With hindsight having done questions 2 and 3, it is also
 clear that the drivers with missing age data were all over 50 years
 old. Given that the df_missing dataset simply didn't have this data,
 it was impossible to know this without looking at the original data
 first. This leaves 2 possibilities:
 1. All ages above 50 were removed, creating a situation where all
 the ages of drivers in `list_age_na_drivers` were removed, without
 affecting any other driverrs (since they were below 50 years old).
 2. The ages of drivers in `list_age_na_drivers` were intentionally
 removed, and they were all above 50 years old.
 
 1 is the more likely scenario, but without looking at the original
 data until tackling question 2, I was led to believe that 2 was
 what was happening.

### Milliseconds

Will try the same strategy as was done for age. After all it worked once.

In [32]:
df_milliseconds_na = df_missing.where(F.col('milliseconds').isNull())
display(df_milliseconds_na)

_c0,driverId,driverRef,number,code,forename,surname,dob,nationality,url,age,raceId,lap,position,time,milliseconds
1129,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,950,1,21,NA,null
1130,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,950,2,20,NA,null
1131,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,950,3,20,NA,null
1132,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,950,4,20,NA,null
1133,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,950,5,20,NA,null
1134,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,950,6,20,NA,null
1135,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,950,7,21,NA,null
1136,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,950,8,21,NA,null
1137,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,950,9,19,NA,null
2886,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,910,1,21,NA,null


In [33]:
display(df_milliseconds_na.summary())

summary,_c0,driverId,driverRef,number,code,forename,surname,dob,nationality,url,age,raceId,lap,position,time,milliseconds
count,47251,47251,47251,18531,47251,47251,47251,47251,47251,47251,44678,47251,47251,47251,47251,0
mean,282873.4741910224,337.3749127002603,null,33.68501430036156,null,null,null,null,null,null,35.96982855096468,574.0513005015766,22.42278470296925,19.252883536856363,null,null
stddev,130533.78086568201,384.0752704949198,null,33.094338024920155,null,null,null,null,null,null,7.035442027441691,386.28413178479366,16.18108571840029,1.5193377464160913,null,null
min,1129,1,albers,2,ALB,Adrian,Albers,1958-02-18,American,http://en.wikipedia.org/wiki/Adrian_Sutil,20,1,1,16,NA,null
25%,173531,20,null,9,null,null,null,null,null,null,30,160,9,18,null,null
50%,306739,58,null,19,null,null,null,null,null,null,37,849,19,19,null,null
75%,401539,819,null,53,null,null,null,null,null,null,42,902,34,20,null,null
max,472421,848,zonta,99,\N,Zsolt,van der Garde,1999-11-13,Venezuelan,http://en.wikipedia.org/wiki/Zsolt_Baumgartner,48,1030,77,24,NA,null


In [34]:
# Looks good here too
47251 + 425253 

Out[16]: 472504

Missingness here appears to be related to position. The full df_missing
dataset has positions from 1-24, whereas this dataset, containing only data 
where milliseconds is missing only has positions from 16 to 24.

Other variables appear similar or the same as the full df_missing dataset,
so will look at position first.

In [36]:
df_milliseconds_na_pos_16_filter = df_missing\
  .filter(F.col('position') >= 16)
display(df_milliseconds_na_pos_16_filter)

_c0,driverId,driverRef,number,code,forename,surname,dob,nationality,url,age,raceId,lap,position,time,milliseconds
1129,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,950,1,21,NA,null
1130,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,950,2,20,NA,null
1131,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,950,3,20,NA,null
1132,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,950,4,20,NA,null
1133,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,950,5,20,NA,null
1134,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,950,6,20,NA,null
1135,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,950,7,21,NA,null
1136,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,950,8,21,NA,null
1137,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,950,9,19,NA,null
1138,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,950,10,16,1:44.596,104596


In [37]:
display(df_milliseconds_na_pos_16_filter.summary())

summary,_c0,driverId,driverRef,number,code,forename,surname,dob,nationality,url,age,raceId,lap,position,time,milliseconds
count,84943,84943,84943,35524,84943,84943,84943,84943,84943,84943,79637,84943,84943,84943,84943,37692
mean,282268.90936274914,330.38702423978435,null,32.40001125999324,null,null,null,null,null,null,35.90548363198011,548.8676053353425,24.364044123706485,18.031774248613775,null,97800.82261487849
stddev,131967.98303180822,381.91330128278076,null,31.881218084450705,null,null,null,null,null,null,7.224395670468197,397.1515565492083,16.749126260242214,1.8193315495327496,null,77262.47096534725
min,1129,1,albers,2,ALB,Adrian,Albers,1958-02-18,American,http://en.wikipedia.org/wiki/Adrian_Sutil,20,1,1,16,16:16.243,68061
25%,172764,20,null,9,null,null,null,null,null,null,29,135,10,17,null,83911
50%,307021,58,null,20,null,null,null,null,null,null,37,845,22,18,null,93172
75%,401419,819,null,35,null,null,null,null,null,null,42,910,37,19,null,104547
max,472423,848,zonta,99,\N,Zsolt,van der Garde,1999-11-13,Venezuelan,http://en.wikipedia.org/wiki/Zsolt_Baumgartner,48,1030,77,24,NA,7496893


Unfortunately that didn't work out. The dataset df_milliseconds_na_pos_filter
filtered for positions equal to or more than 16, but this produced a
dataframe almost twice the size of our milliseconds missing dataset, with
some milliseconds values not missing.

To see the difference between the missing and non missing milliseconds values,
I've filtered the dataset further to provide us with a dataframe with positions
equal to or more than 16, and only non-missing milliseconds values

In [39]:
df_milliseconds_filled_pos_filter = df_milliseconds_na_pos_16_filter\
  .where(F.col('milliseconds').isNotNull())

display(df_milliseconds_filled_pos_filter)

_c0,driverId,driverRef,number,code,forename,surname,dob,nationality,url,age,raceId,lap,position,time,milliseconds
1138,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,950,10,16,1:44.596,104596
2211,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,NA,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,8,43,16,1:34.025,94025
2212,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,8,44,16,1:43.429,103429
2213,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,NA,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,8,45,16,1:23.280,83280
2214,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,8,46,16,1:23.307,83307
2215,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,8,47,16,1:23.417,83417
2216,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,8,48,16,1:23.470,83470
2217,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,8,49,16,1:25.365,85365
2218,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,8,50,16,1:24.629,84629
2219,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,8,51,16,1:27.392,87392


In [40]:
display(df_milliseconds_filled_pos_filter.summary())

summary,_c0,driverId,driverRef,number,code,forename,surname,dob,nationality,url,age,raceId,lap,position,time,milliseconds
count,37692,37692,37692,16993,37692,37692,37692,37692,37692,37692,34959,37692,37692,37692,37692,37692
mean,281511.0219409954,321.6269500159185,null,30.99870534926146,null,null,null,null,null,null,35.82325009296605,517.2971187519898,26.79762283773745,16.500981640666453,null,97800.82261487849
stddev,133742.06402474834,379.00876169393905,null,30.44256305268062,null,null,null,null,null,null,7.458209489556054,408.1849172640876,17.12727698627041,0.5940942822638541,null,77262.47096534725
min,1138,1,albers,2,ALB,Adrian,Albers,1958-02-18,American,http://en.wikipedia.org/wiki/Adrian_Sutil,20,1,1,16,16:16.243,68061
25%,172245,20,null,10,null,null,null,null,null,null,29,111,12,16,null,83911
50%,307699,59,null,20,null,null,null,null,null,null,37,348,25,16,null,93172
75%,401064,819,null,33,null,null,null,null,null,null,42,931,40,17,null,104547
max,472423,848,zonta,99,\N,Zsolt,van der Garde,1999-11-13,Venezuelan,http://en.wikipedia.org/wiki/Zsolt_Baumgartner,48,1030,77,19,9:45.712,7496893


Comparing our milliseconds missing data to the non-missing milliseconds data
to the data with both, though all with positions >= 16, we see a further
difference in the positions data. 

1. The means of the positions data is smaller in the dataset where all
milliseconds information is missing than in the combined dataset, which is
smaller than in the dataset containing only non-missing milliseconds data
where positions >= 16.

2. Range for non-missing milliseconds data positions is 16-19, not 16-24.
Meaning all positions >=20 are NA. Testing this theory below.

In [42]:
df_milliseconds_na_pos_20_filter = df_milliseconds_na_pos_16_filter\
  .filter(F.col('position') >= 20)
display(df_milliseconds_na_pos_20_filter.summary())

summary,_c0,driverId,driverRef,number,code,forename,surname,dob,nationality,url,age,raceId,lap,position,time,milliseconds
count,17489,17489,17489,5745,17489,17489,17489,17489,17489,17489,16738,17489,17489,17489,17489,0
mean,284823.2458688318,340.5766481788553,null,31.166579634464753,null,null,null,null,null,null,36.53566734376867,618.3144833895592,19.70764480530619,20.89524844187775,null,null
stddev,121692.02781287374,384.1266267921892,null,33.80638172828929,null,null,null,null,null,null,6.760152352666198,353.79072303510793,15.192705030137015,1.0825408704222177,null,null
min,1129,1,albers,2,ALB,Adrian,Albers,1958-02-18,American,http://en.wikipedia.org/wiki/Adrian_Sutil,20,3,1,20,NA,null
25%,215336,24,null,8,null,null,null,null,null,null,30,215,7,20,null,null
50%,289733,52,null,17,null,null,null,null,null,null,37,852,16,21,null,null
75%,395109,817,null,31,null,null,null,null,null,null,43,885,29,22,null,null
max,472320,848,zonta,99,\N,Zsolt,van der Garde,1999-11-13,Venezuelan,http://en.wikipedia.org/wiki/Zsolt_Baumgartner,48,1030,70,24,NA,null


%md

Looks like I was right about positions >= 20 all being NA. But why are there 
NAs in positions 16-19 for some but not all entries? Looking at the tables
and the plot optinons above, we can ascertain two things:

1. There are more missing values the lower the position (with 1 being the 
highest position).
2. It can't purely be due to any driver-based attributes, since the same 
drivers have missing and non-missing milliseconds values even at positions
16 and greater. It also can't be due to the time attribute, since all entries
with missing time also have missing milliseconds (but I don't think that's
what we're looking for). 

That leaves race, lap and position. Looking at them below.

In [44]:
df_milliseconds_na.groupby('raceId').agg(F.max('position')).count()

Out[22]: 419

In [45]:
df_milliseconds_filled_pos_filter.groupby('raceId').agg(F.max('position')).count()

Out[23]: 432

In [46]:
df_missing.groupby('raceId').agg(F.max('position')).count()

Out[24]: 437

In [47]:
df_missing.groupby('lap').agg(F.max('position')).count()

Out[25]: 78

In [48]:
df_milliseconds_na.groupby('lap').agg(F.max('position')).count()

Out[26]: 77

In [49]:
df_milliseconds_race1_pos_16 = df_missing\
  .filter(F.col('position') >= 16)\
  .filter(F.col('raceId') == 1)
display(df_milliseconds_race1_pos_16)

_c0,driverId,driverRef,number,code,forename,surname,dob,nationality,url,age,raceId,lap,position,time,milliseconds
14422,2,heidfeld,null,HEI,Nick,Heidfeld,1977-05-10,German,http://en.wikipedia.org/wiki/Nick_Heidfeld,42,1,9,17,NA,null
14503,2,heidfeld,null,HEI,Nick,Heidfeld,1977-05-10,German,http://en.wikipedia.org/wiki/Nick_Heidfeld,42,1,3,17,1:31.697,91697
14554,2,heidfeld,null,HEI,Nick,Heidfeld,1977-05-10,German,http://en.wikipedia.org/wiki/Nick_Heidfeld,42,1,1,17,2:37.754,157754
14571,2,heidfeld,null,HEI,Nick,Heidfeld,1977-05-10,German,http://en.wikipedia.org/wiki/Nick_Heidfeld,42,1,4,17,1:31.867,91867
14585,2,heidfeld,null,HEI,Nick,Heidfeld,1977-05-10,German,http://en.wikipedia.org/wiki/Nick_Heidfeld,42,1,8,17,1:31.801,91801
14602,2,heidfeld,null,HEI,Nick,Heidfeld,1977-05-10,German,http://en.wikipedia.org/wiki/Nick_Heidfeld,42,1,10,17,1:33.063,93063
14762,2,heidfeld,null,HEI,Nick,Heidfeld,1977-05-10,German,http://en.wikipedia.org/wiki/Nick_Heidfeld,42,1,7,17,1:31.659,91659
14800,2,heidfeld,null,HEI,Nick,Heidfeld,1977-05-10,German,http://en.wikipedia.org/wiki/Nick_Heidfeld,42,1,2,17,1:37.640,97640
14883,2,heidfeld,null,HEI,Nick,Heidfeld,1977-05-10,German,http://en.wikipedia.org/wiki/Nick_Heidfeld,42,1,5,17,1:32.234,92234
15011,2,heidfeld,null,HEI,Nick,Heidfeld,1977-05-10,German,http://en.wikipedia.org/wiki/Nick_Heidfeld,42,1,14,17,1:31.408,91408


My suspicion was that the last 5 positions of each race were knocked off.
This explains why 20-24 were always empty, with more missing values the closer
a position is to 20.

I decided to look more closely at a specific race - race 1. This quickly 
disproved my hypothesis. Race 1 contains positions up to 19, but one laptime
for position 18 (Adrian Sutil), in lap 8, was recorded.

This eliminates a strict raceId + position based removal, because the same 
race has missing and non-missing values for the same driver, having the same
final position for the race.

A lap based explanation also seems implausible. The dataset
containing only missing milliseconds data has all but one of the laps 
represented. From the summary data, we can see that this is the maximum lap
number, lap 78. This is probably absent because people in the final positions
crash out and never get to that last lap, meaning the whole row is missing
instead of just the millisecond data. Incompletions are very common in F1.
So, the missingness pattern has little  to do with the lap number. 

The final possibility I can think of is lap + position, where earlier positions
have less laptimes taken off, and later positions have more laptimes taken off.
While this has already been established to be true from the bargraph of 
position and number of non-missing milliseconds data, I don't see a clear rule
to decide which laps are missed out. Race 1, Position 17 has missing values 
for lap 9, 13, 19, among others, whereas position 18 is missing laps 1-7, 
9-21, 23 and 24.

It seems completely arbitrary to me. At this point, it seems like the pattern
of missing milliseconds data is weighted probabilistically according to position
from the range of 16-19. Milliseconds data is completely absent beyond 20, and 
all milliseconds data is available for positions above 15.

Moving on.

### Nationality

In [52]:
df_nationality_na = df_missing.filter(F.col('nationality') == 'NA')
display(df_nationality_na)

_c0,driverId,driverRef,number,code,forename,surname,dob,nationality,url,age,raceId,lap,position,time,milliseconds
12,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,NA,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,37,28,2,1:37.673,97673
62,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,NA,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,841,4,2,1:32.582,92582
67,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,NA,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,841,9,2,1:32.612,92612
79,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,NA,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,841,21,2,1:31.212,91212
98,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,NA,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,841,40,2,1:31.200,91200
120,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,NA,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,842,18,2,1:43.388,103388
125,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,NA,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,842,23,2,1:43.009,103009
128,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,NA,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,842,26,5,1:43.322,103322
141,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,NA,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,842,39,5,1:43.305,103305
152,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,NA,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,979,4,5,2:09.188,129188


In [53]:
display(df_nationality_na.summary())

summary,_c0,driverId,driverRef,number,code,forename,surname,dob,nationality,url,age,raceId,lap,position,time,milliseconds
count,47112,47112,47112,22915,47112,47112,47112,47112,47112,47112,40914,47112,47112,47112,47112,42431
mean,236831.90064102566,229.8604177279674,null,25.375561859044293,null,null,null,null,null,null,36.9850417949846,478.89633214467653,29.8841059602649,9.670168959076245,null,94927.01703942871
stddev,136309.13495101867,345.22003963236295,null,25.059337728224083,null,null,null,null,null,null,7.277806101583822,401.7174307637275,18.453123903650155,5.565508413514029,null,53836.48933054257
min,12,1,albers,2,ALB,Adrian,Albers,1958-02-18,NA,http://en.wikipedia.org/wiki/Adrian_Sutil,20,1,1,1,19:16.450,67377
25%,118420,14,null,8,null,null,null,null,null,null,32,112,14,5,null,81868
50%,237628,30,null,19,null,null,null,null,null,null,38,227,29,9,null,90279
75%,354286,807,null,28,null,null,null,null,null,null,43,910,44,14,null,102057
max,472486,848,zonta,99,\N,Zsolt,van der Garde,1999-11-13,NA,http://en.wikipedia.org/wiki/Zsolt_Baumgartner,48,1030,78,24,NA,3681464


In [54]:
# 47112 is the same number we saw in the groupby above, so we should be good

In [55]:
display(df_missing.summary())

summary,_c0,driverId,driverRef,number,code,forename,surname,dob,nationality,url,age,raceId,lap,position,time,milliseconds
count,472504,472504,472504,230938,472504,472504,472504,472504,472504,472504,411348,472504,472504,472504,472504,425253
mean,236252.5,229.7737775764861,null,25.328412820757087,null,null,null,null,null,null,36.98124702198625,480.07822156002914,29.871804259858116,9.651368030746829,null,94899.58912224017
stddev,136400.3001340784,345.3821632369074,null,24.985651553593236,null,null,null,null,null,null,7.251128022983076,402.0313510566064,18.38969559962932,5.562433907092902,null,66244.43117416694
min,1,1,albers,2,ALB,Adrian,Albers,1958-02-18,American,http://en.wikipedia.org/wiki/Adrian_Sutil,20,1,1,1,15:25.994,66957
25%,118129,14,null,8,null,null,null,null,null,null,32,112,14,5,null,81931
50%,236205,30,null,19,null,null,null,null,null,null,38,228,29,9,null,90364
75%,354341,807,null,28,null,null,null,null,null,null,43,911,44,14,null,102028
max,472504,848,zonta,99,\N,Zsolt,van der Garde,1999-11-13,Venezuelan,http://en.wikipedia.org/wiki/Zsolt_Baumgartner,48,1030,78,24,NA,7507547


Again this cannot be driver based - we see here a lot of missing values for
Lewis Hamilton, but above we saw some entries with Lewis Hamilton listed as
British. Since the same driver has both missing and non-missing values, a pure
driver-based criterion for missing data is ruled out.

That leaves raceId, lap, position, and timing data. I am not optimistic
here either, since the descriptives for these are almost identical to the 
whole df_missing dataset. This suggests that the df_nationality_na dataset
is randomly sampled from the full df_missing dataset, and hence that the
nationality missingness was randomly imposed.

Still I think I should have a look.

In [57]:
df_missing.groupby('raceId').agg(F.max('position')).count()

Out[32]: 437

In [58]:
df_nationality_na.groupby('raceId').agg(F.max('position')).count()

Out[33]: 437

In [59]:
df_missing.groupby('lap').agg(F.max('position')).count()

Out[34]: 78

In [60]:
df_nationality_na.groupby('lap').agg(F.max('position')).count()

Out[35]: 78

In [61]:
df_missing.groupby('position').agg(F.max('position')).count()

Out[36]: 24

In [62]:
df_nationality_na.groupby('position').agg(F.max('position')).count()

Out[37]: 24

All races, positions, and laps are represented in our nationality-missing 
dataset. There is also entries where nationality is missing for both 
valid and missing millisecond data.

These suggest that any "@ this position/lap/race, remove nationality"
isn't what is happening.

There could be a more complex hard and/or probabilistic rule "@ this
position, remove nationality for this lap/race/driver/timing range with
this probability". But that's going to be awfully difficult to see from data
exploration or even visualisation, especially given, the high number of laps
and races.

I think it's reasonable to assume that the nationality missingness is randomly
imposed, particularly given the lack of bias in the distributions of any other 
variable in the dataset.

## 2. Plot the distributions of variables with missing values against the observed ones and explain how different missingness patterns are affecting the distributions.

## Prep data
1. Get original data
2. Use the fill_dummy_values function to prepare the data for the plot
3. toPandas() - don't want to have to deal with matplotlib + pyspark

In [66]:
df_laptimes = spark.read.csv('/mnt/ne-gr5069/raw/lap_times.csv', header = True)
df_drivers = spark.read.csv('/mnt/ne-gr5069/raw/drivers.csv', header = True)

df_drivers_with_age = df_drivers\
  .withColumn("age", F.datediff(F.current_date(),F.col("dob"))/365.25)
df_drivers_with_age = df_drivers_with_age\
  .withColumn("age", df_drivers_with_age["age"].cast(IntegerType()))

df_original = df_laptimes.join(df_drivers_with_age, on = ['driverId'])
display(df_original.summary())

summary,driverId,raceId,lap,position,time,milliseconds,driverRef,number,code,forename,surname,dob,nationality,url,age
count,472504,472504,472504,472504,472504,472504,472504,472504,472504,472504,472504,472504,472504,472504,472504
mean,229.7737775764861,480.07822156002914,29.871804259858116,9.651368030746829,null,95554.85977473206,null,25.328412820757087,null,null,null,null,null,null,39.119495284696
stddev,345.38216323690835,402.0313510566062,18.38969559962933,5.5624339070928865,null,71232.1852395675,null,24.985651553593197,null,null,null,null,null,null,8.728104640739067
min,1,1,1,1,10:32.179,100000,albers,10,ALB,Adrian,Albers,1958-02-18,American,http://en.wikipedia.org/wiki/Adrian_Sutil,20
25%,14.0,112.0,14.0,5.0,null,82237.0,null,8.0,null,null,null,null,null,null,32
50%,30.0,228.0,29.0,9.0,null,90881.0,null,19.0,null,null,null,null,null,null,38
75%,807.0,911.0,44.0,14.0,null,102661.0,null,28.0,null,null,null,null,null,null,47
max,9,999,9,9,9:45.712,99999,zonta,\N,\N,Zsolt,van der Garde,1999-11-13,Venezuelan,http://en.wikipedia.org/wiki/Zsolt_Baumgartner,62


In [67]:
df_full = df_missing.join(df_original.select(F.col('driverId'),
                                             F.col('raceId'),
                                             F.col('lap'),
                                             F.col('nationality').alias('og_nat'),
                                             F.col('milliseconds').alias('og_ms'),
                                             F.col('age').alias('og_age')
                                            ), on = ['driverId', 'raceId', 'lap']
                         )

df_full = df_full.withColumn("og_ms", df_full["og_ms"].cast(IntegerType()))
display(df_full)

driverId,raceId,lap,_c0,driverRef,number,code,forename,surname,dob,nationality,url,age,position,time,milliseconds,og_nat,og_ms,og_age
1,5,37,10561,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,9,1:24.159,84159,British,84159,35
1,9,7,13501,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,20,NA,null,British,103434,35
1,9,23,12453,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,19,NA,null,British,97195,35
1,10,23,6656,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,1,1:24.557,84557,British,84557,35
1,13,35,827,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,5,1:48.433,108433,British,108433,35
1,21,42,9082,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,3,1:22.313,82313,British,82313,35
1,25,49,11615,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,10,1:17.828,77828,British,77828,35
1,26,25,13634,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,1,1:34.946,94946,British,94946,35
1,27,56,6150,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,3,1:16.879,76879,British,76879,35
1,33,20,12380,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35,15,1:21.772,81772,British,81772,35


In [68]:
display(df_full.summary())

summary,driverId,raceId,lap,_c0,driverRef,number,code,forename,surname,dob,nationality,url,age,position,time,milliseconds,og_nat,og_ms,og_age
count,472504,472504,472504,472504,472504,230938,472504,472504,472504,472504,472504,472504,411348,472504,472504,425253,472504,472504,472504
mean,229.7737775764861,480.07822156002914,29.871804259858116,236252.5,null,25.328412820757087,null,null,null,null,null,null,36.98124702198625,9.651368030746829,null,94899.58912224017,null,95554.85977473206,39.119495284696
stddev,345.3821632369095,402.0313510566072,18.389695599629317,136400.30013407842,null,24.985651553593183,null,null,null,null,null,null,7.251128022983017,5.562433907092896,null,66244.4311741669,null,71232.18523956732,8.72810464073903
min,1,1,1,1,albers,2,ALB,Adrian,Albers,1958-02-18,American,http://en.wikipedia.org/wiki/Adrian_Sutil,20,1,15:25.994,66957,American,66957,20
25%,14,112,14,118538,null,8,null,null,null,null,null,null,32,5,null,81931,null,82244,32
50%,30,228,29,236242,null,19,null,null,null,null,null,null,38,9,null,90363,null,90900,38
75%,807,911,44,354514,null,28,null,null,null,null,null,null,43,14,null,102067,null,102676,47
max,848,1030,78,472504,zonta,99,\N,Zsolt,van der Garde,1999-11-13,Venezuelan,http://en.wikipedia.org/wiki/Zsolt_Baumgartner,48,24,NA,7507547,Venezuelan,7507547,62


In [69]:
df_full_pd = df_full.toPandas()
df_full_dummy_vals = fill_dummy_values(df_full_pd, 0.075)

/local_disk0/tmp/1583455334968-0/PythonShell.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

## Age

In [71]:
#Get missing values for coloring
null_age = df_full_pd['age'].isnull()

#Generate Scatter plot
fig_age, ax = plt.subplots()
df_full_dummy_vals.plot(x = 'age',
                        y = 'og_age',
                        kind = 'scatter',
                        alpha = 0.5,
                        c = null_age,
                        cmap = 'rainbow',
                        figsize=(12,8),
                        grid = True,
                        legend = True,
                        ax = ax)
ax.legend(['Not Missing'])

Out[42]: <matplotlib.legend.Legend at 0x7f00e87e71d0>

In [72]:
fig_age_NA_compare = plt.figure(figsize = (12,10))
sns.distplot(df_full_pd['age'], hist = False, kde = True,
             color = 'red', 
             kde_kws={'linewidth': 2})
sns.distplot(df_full_pd['og_age'], hist = False, kde = True,
             color = 'blue', 
             kde_kws={'linewidth': 2})
display(fig_age_NA_compare)

In [73]:
display(fig_age)

As a result of the missingness pattern in the age data, the overall
distribution of ages in df_missing is much younger. In the missing data,
there was no data above age 50.

There were some minor differences in how I calculated age from the
df_missing dataset (I used the current day's timestamp, subtracting
DOB). Nevertheless, for ages that were available, differences were 
minor (see scatterplot), suggesting this computational differences
would not be sufficient to account for significantly larger ages of
those whose ages were missing from df_missing.

## Milliseconds

In [76]:
#Get missing values for coloring
null_ms = df_full_pd['milliseconds'].isnull()

#Generate Scatter plot
fig_ms, ax = plt.subplots()
df_full_dummy_vals.plot(x = 'milliseconds',
                        y = 'og_ms',
                        kind = 'scatter',
                        alpha = 0.5,
                        c = null_ms,
                        cmap = 'rainbow',
                        figsize=(12,8),
                        grid = True,
                        legend = True,
                        ax = ax)
ax.legend(['Not Missing'])

Out[45]: <matplotlib.legend.Legend at 0x7f00e86b6240>

In [77]:
fig_ms_NA_compare = plt.figure(figsize = (12,10))
sns.distplot(df_full_pd['og_ms'], hist = False, kde = True,
             color = 'blue', 
             kde_kws={'linewidth': 2})
sns.distplot(df_full_pd['milliseconds'], hist = False, kde = True,
             color = 'red', 
             kde_kws={'linewidth': 2})
display(fig_ms_NA_compare)


In [78]:
display(fig_ms)

The overall distributions in laptimes seems very similar between the 
original observed data and the data contained in df_missing. In the 
density plot, the two curves overlap to the point where distinguishing 
them becomes difficult. The full data has slightly higher peaks, with
the most prominent at the first and highest peak near 0. I don't think
this is indicative of a skew in the df_missing laptimes data towards
faster laps, despite how missingness in the laptime data was related to
race position, as the higher blue peaks are true for all of the peaks,
and the scatterplot of original against df_missing data appears to have
a very similar distribution. I think it is more likely that the 
higher peaks reflect a greater frequency of data in the full dataset at
all points in the distribution, since it also includes data that was 
missing in the df_missing dataset.

The overall distribution of laptimes in df_missing thus does not seem
affected by the pattern of missingness in the df_missing dataset for
the milliseconds column.

## Nationality

In [81]:
nat_count = pd.DataFrame(df_full_pd['nationality'].value_counts())
nat_og_count = pd.DataFrame(df_full_pd['og_nat'].value_counts())
nat_count_df = nat_count.join(nat_og_count)

In [82]:
display(nat_count_df.plot(kind = 'bar'))

In [83]:
display(nat_count_df.plot.scatter(x = 'nationality',
                                  y = 'og_nat'))

The overall frequency distribution of nationality in the full data and 
df_missing seem very similar. This is also seen in the perfectly linear
relationship between the two distributions in the scatterplot. 

Like with the laptimes data, the peaks
for each nationality is higher than the df_missing data, but that is likely 
due to the inclusion of entries that were missing in the df_missing dataset.

The similar shapes of the distributions suggests that the overall frequency
distribution of nationalities in df_missing was not affected by the pattern of
missingness in the df_missing dataset for the nationalities column. This also
somewhat supports the notion that the pattern of missingness in this column
was completely randomly imposed.

## 3. Based on your findings, use at least two different imputation techniques to impute the variables for further analysis. Assess the effectiveness of imputation techniques both visually and analytically (e.g. using an appropriate accuracy metric that you defined).

### KNN imputation

In [87]:
df_missing_for_impute = df_missing.toPandas()
df_missing_for_impute = df_missing_for_impute[['driverId',
                                               'age',
                                               'raceId',
                                               'nationality',
                                               'lap',
                                               'position',
                                               'milliseconds']]

df_missing_for_impute = df_missing_for_impute.replace('NA', np.nan)

In [88]:
#Encode nationality as ordinal variable to run KNN imputer
encoder = OrdinalEncoder()
encoded_nat = encode(df_missing_for_impute['nationality'])
encoder.categories_

/databricks/python/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
 self._setitem_with_indexer(indexer, value)
Out[52]: [array(['American', 'Argentine', 'Australian', 'Austrian', 'Belgian',
 'Brazilian', 'British', 'Canadian', 'Colombian', 'Czech', 'Danish',
 'Dutch', 'Finnish', 'French', 'German', 'Hungarian', 'Indian',
 'Indonesian', 'Irish', 'Italian', 'Japanese', 'Malaysian',
 'Mexican', 'Monegasque', 'New Zealander', 'Polish', 'Portuguese',
 'Russian', 'Spanish', 'Swedish', 'Swiss', 'Thai', 'Venezuelan'],
 dtype=object)]

In [89]:
df_missing_for_impute['nationality'] = encoded_nat
df_missing_for_impute.head(15)

,driverId,age,raceId,nationality,lap,position,milliseconds
0,1,35.0,37,6,17,2,97663.0
1,1,35.0,37,6,18,2,97579.0
2,1,35.0,37,6,19,1,97720.0
3,1,35.0,37,6,20,1,100973.0
4,1,35.0,37,6,21,3,120933.0
5,1,35.0,37,6,22,3,96701.0
6,1,35.0,37,6,23,2,97257.0
7,1,35.0,37,6,24,2,97907.0
8,1,35.0,37,6,25,2,98180.0
9,1,35.0,37,6,26,2,97822.0


In [90]:
#Run KNNImputer
# This takes very long to run, so I've run it and saved the output in S3. 
# Will continue the assignment by reading this in.

#knn_imputer = KNNImputer(n_neighbors=10)
#fill missing values by imputing
#array_missing_imputed = knn_imputer.fit_transform(df_missing_for_impute)
#df_missing_imputed = pd.DataFrame(array_missing_imputed)
#spark_df_missing_imputed = spark.createDataFrame(df_missing_imputed)
#spark_df_missing_imputed.coalesce(1).write.csv('/mnt/xql2001-gr5069/interim/assignment_2/spark_df_missing_imputed.csv')

In [91]:
df_missing_imputed = spark.read.csv('/mnt/xql2001-gr5069/interim/assignment_2/spark_df_missing_imputed.csv', header = False)
display(df_missing_imputed)

_c0,_c1,_c2,_c3,_c4,_c5,_c6
1.0,35.0,37.0,6.0,17.0,2.0,97663.0
1.0,35.0,37.0,6.0,18.0,2.0,97579.0
1.0,35.0,37.0,6.0,19.0,1.0,97720.0
1.0,35.0,37.0,6.0,20.0,1.0,100973.0
1.0,35.0,37.0,6.0,21.0,3.0,120933.0
1.0,35.0,37.0,6.0,22.0,3.0,96701.0
1.0,35.0,37.0,6.0,23.0,2.0,97257.0
1.0,35.0,37.0,6.0,24.0,2.0,97907.0
1.0,35.0,37.0,6.0,25.0,2.0,98180.0
1.0,35.0,37.0,6.0,26.0,2.0,97822.0


In [92]:
pd_missing_imputed = df_missing_imputed.toPandas()
pd_missing_imputed = pd_missing_imputed.rename(columns={"_c0": "driverId",
                                                        "_c1": "age",
                                                        "_c2": "raceId",
                                                        "_c3": "nationality",
                                                        "_c4": "lap",
                                                        "_c5": "position",
                                                        "_c6": "milliseconds"}
                                              )
pd_missing_imputed = pd_missing_imputed.astype('float64', copy=False)
pd_missing_imputed['nationality'] = np.round(pd_missing_imputed['nationality'])

nat_dict = {0: 'American',
            1: 'Argentine',
            2: 'Australian',
            3: 'Austrian',
            4: 'Belgian',
            5: 'Brazilian',
            6: 'British',
            7: 'Canadian',
            8: 'Colombian',
            9: 'Czech',
            10: 'Danish',
            11: 'Dutch',
            12: 'Finnish',
            13: 'French',
            14: 'German',
            15: 'Hungarian',
            16: 'Indian',
            17: 'Indonesian',
            18: 'Irish',
            19: 'Italian',
            20: 'Japanese',
            21: 'Malaysian',
            22: 'Mexican',
            23: 'Monegasque',
            24: 'New Zealander',
            25: 'Polish',
            26: 'Portuguese',
            27: 'Russian',
            28: 'Spanish',
            29: 'Swedish',
            30: 'Swiss',
            31: 'Thai',
            32: 'Venezuelan'
           }

pd_missing_imputed['nationality'] = pd_missing_imputed['nationality'].map(nat_dict)
pd_missing_imputed.head(15)

,driverId,age,raceId,nationality,lap,position,milliseconds
0,1.0,35.0,37.0,British,17.0,2.0,97663.0
1,1.0,35.0,37.0,British,18.0,2.0,97579.0
2,1.0,35.0,37.0,British,19.0,1.0,97720.0
3,1.0,35.0,37.0,British,20.0,1.0,100973.0
4,1.0,35.0,37.0,British,21.0,3.0,120933.0
5,1.0,35.0,37.0,British,22.0,3.0,96701.0
6,1.0,35.0,37.0,British,23.0,2.0,97257.0
7,1.0,35.0,37.0,British,24.0,2.0,97907.0
8,1.0,35.0,37.0,British,25.0,2.0,98180.0
9,1.0,35.0,37.0,British,26.0,2.0,97822.0


#### Age

In [94]:
fig_age_imp_compare = plt.figure(figsize = (12,10))
sns.distplot(pd_missing_imputed['age'], hist = False, kde = True,
             color = 'red', 
             kde_kws={'linewidth': 2})
sns.distplot(df_full_pd['og_age'], hist = False, kde = True,
             color = 'blue', 
             kde_kws={'linewidth': 2})
display(fig_age_imp_compare)

In [95]:
missing_age_index = df_full_pd['age'].isna()
predicted_age_list = pd_missing_imputed['age'][missing_age_index]
actual_age_list = df_full_pd['og_age'][missing_age_index]
rmse_age = RMSE(predicted_age_list, actual_age_list)
rmse_age

Out[75]: 16.88799241245684

The KNN imputer was largely unsuccessful at imputing the missing values for 
age. The distribution of the data filled with the imputed values largely
resembles the distribution of ages in df_missing before imputation.
The distribution of ages remains younger than the actual data. One redeeming
factor of the KNN model is a higher peak at the high 40s in the imputed data
compared to the df_missing data, suggesting that the model successfully 
inferred that the drivers with missing age data were older. However, without
exposure to ages beyond 50, it was unable to impute the ages of these drivers
to be as old as they actually are.

Quantitatively, the limited effectiveness of the KNN imputer can be seen in
the RMSE value of 16.9, which is high for a distribution with a range of only
about 40.

#### Milliseconds

In [98]:
fig_ms_imp_compare = plt.figure(figsize = (12,10))
sns.distplot(df_full_pd['og_ms'], hist = False, kde = True,
             color = 'blue', 
             kde_kws={'linewidth': 2})
sns.distplot(pd_missing_imputed['milliseconds'], hist = False, kde = True,
             color = 'red', 
             kde_kws={'linewidth': 2})
display(fig_ms_imp_compare)

In [99]:
missing_ms_index = df_full_pd['milliseconds'].isna()
predicted_ms_list = pd_missing_imputed['milliseconds'][missing_age_index]
actual_ms_list = df_full_pd['og_ms'][missing_age_index]
rmse_ms = RMSE(predicted_ms_list, actual_ms_list)
rmse_ms

Out[157]: 72906.63581083345

At first glance, the KNN imputer seemed largely successful in imputing 
missing laptime data. This is seen in the RMSE of 72906, which is small for a
 variable with a range in the millions. In the plot, we see that the 
imputation created a skew in laptimes towards faster laps, imputing more 
missing values as having lower values than was actually observed. This was 
likely due to the disproportionate volume of fast laps in the dataset. This
effect does not seem to be too severe given the small RMSE value and the fact 
that the imputed and actual data density plots are still very similar in 
shape.

Nevertheless, by creating the skew, the imputer created a laptime distribution
that is less similar to the distribution of the full data than the df_missing
laptime distribution. Rather than having a skew towards faster laps, however
minor, the df_missing laptime distribution was very similar to that of the full
data.

By shifting the distribution of missing data to be less similar to the full
data than the actual distribution, the KNN imputer proved to be less effective
than just removing missing values, at least for the case of representing the
distribution of F1 laptimes.

#### Nationality

In [102]:
fig_nat_imp_compare = plt.figure(figsize = (12,10))
sns.countplot(y = 'nationality',
              data=pd_missing_imputed,
              color = 'red',
              alpha = 0.5)
sns.countplot(y = 'og_nat',
              data=df_full_pd,
              color = 'blue',
              alpha = 0.5)
display(fig_nat_imp_compare)

In [103]:
missing_nat_index = df_full_pd['nationality'] == 'NA'
predicted_nat_list = pd_missing_imputed['nationality'][missing_nat_index]
actual_nat_list = df_full_pd['og_nat'][missing_nat_index]
confusion_matrix_nat = confusion_matrix(actual_nat_list, predicted_nat_list)
nat_accuracy = np.diagonal(confusion_matrix_nat).sum()/missing_nat_index.sum()
nat_accuracy

Out[174]: 0.08450076413652573

In [104]:
fig_nat_cm = plt.figure(figsize = (10,10))
sns.heatmap(confusion_matrix_nat)
display(fig_nat_cm)

The KNN imputer did a devastatingly poor job at imputing the missing 
nationalities data. This can be seen from:

1. The dismal accuracy of 0.08 (though to be fair, chance is about 3%, so
the model is better than chance, but beating chance is very far from what
we need for missing values imputation).
2. The density plot, where no nationality was at all well predicted, except
maybe for British and Germans.
2. The heatmap of the confusion matrix (should have high counts on the 
diagonals, which is only somewhat true).


The imputer predicted less members of the frequent entries, like British 
and German, and more members of the infrequent nationalities such as Irish and
Danish. However, the model also tended to miss out other infrequent 
nationalities like Finnish and Brazillian.

This was likely because with the KNN imputer, we had to code nationality as
ordinal data, when it is in fact nominal. OneHotEncoding is more appropriate
for nominal data, but since it creates features based on membership to a
category, missing data would not appear to be missing under OneHotEncoding,
they would just appear as not a member of any class, leaving nothing to 
impute. The fact that driverId and raceId were also actually nominal features
but were included as continuous features were also likely contributors to the
issue.

*Honestly I didn't anticipate that this would be too much of a problem, and*
*have now run out of time to retrain new models for imputation, so I'm going*
*with what I have. In my defense, properly encoding driver and race Id probably*
*would have created a better model but also would've meant creating about 2000*
*new features. That would have made model training take a lot longer than I*
*have time for.*

### Mean/Modal imputation

In [107]:
df_mean_mode_impute = df_full_pd.copy(deep = True)
df_mean_mode_impute['age'].fillna(df_mean_mode_impute['age'].mean(),
                                  inplace = True)
df_mean_mode_impute['milliseconds'].fillna(df_mean_mode_impute['milliseconds'].mean(),
                                           inplace = True)

mode_nationality = "German"
df_mean_mode_impute['nationality'] = df_mean_mode_impute['nationality'].str.replace('NA', mode_nationality)
df_mean_mode_impute.head(20)

,driverId,raceId,lap,_c0,driverRef,number,code,forename,surname,dob,nationality,url,age,position,time,milliseconds,og_nat,og_ms,og_age
0,1,5,37,10561,hamilton,44.0,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35.0,9,1:24.159,84159.000000,British,84159,35
1,1,9,7,13501,hamilton,44.0,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35.0,20,NA,94899.589122,British,103434,35
2,1,9,23,12453,hamilton,44.0,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35.0,19,NA,94899.589122,British,97195,35
3,1,10,23,6656,hamilton,44.0,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35.0,1,1:24.557,84557.000000,British,84557,35
4,1,13,35,827,hamilton,44.0,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35.0,5,1:48.433,108433.000000,British,108433,35
5,1,21,42,9082,hamilton,44.0,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35.0,3,1:22.313,82313.000000,British,82313,35
6,1,25,49,11615,hamilton,44.0,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35.0,10,1:17.828,77828.000000,British,77828,35
7,1,26,25,13634,hamilton,44.0,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35.0,1,1:34.946,94946.000000,British,94946,35
8,1,27,56,6150,hamilton,44.0,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35.0,3,1:16.879,76879.000000,British,76879,35
9,1,33,20,12380,hamilton,44.0,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,35.0,15,1:21.772,81772.000000,British,81772,35


#### Age

In [109]:
fig_age_mean_imp_compare = plt.figure(figsize = (12,10))
sns.distplot(df_mean_mode_impute['age'], hist = False, kde = True,
             color = 'red', 
             kde_kws={'linewidth': 2})
sns.distplot(df_full_pd['og_age'], hist = False, kde = True,
             color = 'blue', 
             kde_kws={'linewidth': 2})
display(fig_age_mean_imp_compare)

In [110]:
predicted_age_list_mean_imp = df_mean_mode_impute['age'][missing_age_index]
rmse_age_mean_imp = RMSE(predicted_age_list_mean_imp, actual_age_list)
rmse_age_mean_imp

Out[155]: 16.29005335432163

The mean imputation was largely unsuccessful at imputing the missing values for 
age. Like the KNN imputer, the distribution of the data filled with the imputed
values largely resembles the distribution of ages in df_missing before
imputation, save for the tall peak at the mean age of about 37. Visually, the
mean imputation seems to have performed more poorly than KNN imputation given
the lower resemblance to the original distribution, due to the tall peak.
This is despite a slightly lower RMSE value for mean imputation  at 16.3 
instead of 16.9. Nevertheless, 16.2 remains high for a distribution with a 
range of only about 40. 

Like the KNN imputed distribution, the distribution of ages remains younger
than the actual data. Using only the mean, the mean imputation could not 
rightly infer that the drivers with missing ages were older, unlike the KNN
imputation.

Overall, the mean imputation was unsuccessful, and seemed to perform worse
than KNN imputation for filling in the missing age date.

#### Milliseconds

In [113]:
fig_ms_mean_imp_compare = plt.figure(figsize = (12,10))
sns.distplot(df_full_pd['og_ms'], hist = False, kde = True,
             color = 'blue', 
             kde_kws={'linewidth': 2})
sns.distplot(df_mean_mode_impute['milliseconds'], hist = False, kde = True,
             color = 'red', 
             kde_kws={'linewidth': 2})
display(fig_ms_mean_imp_compare)

In [114]:
predicted_ms_list_mean_imp = df_mean_mode_impute['milliseconds'][missing_age_index]
rmse_ms_mean_imp = RMSE(predicted_ms_list_mean_imp, actual_ms_list)
rmse_ms_mean_imp

Out[186]: 8941.524095164037

Overall, the mean imputation seems successful at imputing missing values for
laptimes.

Quantitatively, the RMSE value of 8941 is very small compared to the range
of over 7 million. Visually, the mean imputed distribution lacks the skew
at around 500k milliseconds that was present in the KNN imputed distribution,
closely adhering to the original distribution. 

However, there are two notable deviations. 

First, the prominent blue peak at about 100000ms. This is more pronounced in 
this plot than in the plot from question 2 comparing the df_missing laptime
data to the full data, despite identical settings. This is very odd given
that mean imputation should have increased, not decreased the number of values 
at 100000ms, which is about the mean value that was imputed (about 94k).
After spending quite some time checking if my code was correct, I think it
probably was, and what we see is probably related to the smoothing of the plot
rather than changes in the distribution.

Second, blue peaks popping out over the red line at all peaks across the 
distributionshows that the mean imputation was imperfect at restoring the data
to its original state, with less red values across the range being similar to 
the plot from question 2 comparing the df_missing laptime data to the full
data.

#### Nationality

In [117]:
fig_nat_mode_imp_compare = plt.figure(figsize = (12,10))
sns.countplot(y = 'nationality',
              data=df_mean_mode_impute,
              color = 'red',
              alpha = 0.5)
sns.countplot(y = 'og_nat',
              data=df_full_pd,
              color = 'blue',
              alpha = 0.5)
display(fig_nat_mode_imp_compare)

In [118]:
predicted_nat_mode_imp_list = df_mean_mode_impute['nationality'][missing_nat_index]
confusion_matrix_mode_imp_nat = confusion_matrix(actual_nat_list, predicted_nat_mode_imp_list)
nat_mode_imp_accuracy = (np.diagonal(confusion_matrix_mode_imp_nat).sum())/missing_nat_index.sum()
nat_mode_imp_accuracy

Out[179]: 0.17723722193920868

In [119]:
fig_nat_mode_imp_cm = plt.figure(figsize = (10,10))
sns.heatmap(confusion_matrix_mode_imp_nat)
display(fig_nat_mode_imp_cm)

The modal imputer did a poor job at imputing the missing nationalities data.
This can be seen from:

1. The accuracy of 0.17, which is much better than chance at 0.03125, but far
from what we need as an imputer. That being said, modal imputation outperformed
the KNN imputer.
2. The density plot, which predictably overestimated the modal nationality
(German) while leaving every other nationality lacking in representation
in the data.
2. The heatmap of the confusion matrix (the colourful strip is German), and
the diagonal is pretty much black when it should be white.

In [122]:
#Unmount S3 buckets
#dbutils.fs.unmount("/mnt/%s" % MOUNT_NAME_RAW)
#dbutils.fs.unmount("/mnt/%s" % MOUNT_NAME_PROC)